# 3.4 Aprendizaje supervisado: Naive Bayes

Profesor: Juan Ramón Rico (<juanramonrico@ua.es>)

## Resumen
---

Se presentarán los principios básicos de los algoritmos Bayesianos, concretamente el conocido como Naive Bayes.

Parte de estos ejemplos están basados en los ejemos de https://www.geeksforgeeks.org/naive-bayes-classifiers/

In [2]:
import numpy as np
import pandas as pd

# Ejemplo de cálculo de probabilidades con Naive Bayes

Teorema de Bayes:

- La probabilidad de un evento $A$ condicionado a otro evento $B$ se puede calcular como: $$P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(B|A) P(A)}{P(B)}$$ donde $P(A \cap B)$ es la probabilidad de que $A$ y $B$ ocurran a la vez, y $P(B)$ denota la probabilidad de $B$.

- En clasificación asumimos independencia entre variables y la normalización la fórmula es: $$P(y | x_1, x_2,\dots,x_n) = \frac{P(y) \prod_{n=1}^{n} P(x_i|y)}{\sum_j P(y_j) \prod_{n=1}^{n} P(x_i|y_j)}$$ siendo $y$ una clase de la variable objetivo, $P(y)$ la probabilidad de la clase $y$ y $P(x_i|y)$ la probabilidad condicional de $x_i$ dado $y$. La clase sería la que consiguiera mayor probabilidad: $$y = argmax_{y_j} \; P(y_j | x_1, x_2,\dots,x_n)$$


In [3]:
## Datos

path = 'https://www.dlsi.ua.es/~juanra/UA/datasets'
path_tennis = f'{path}/tennis-en.csv'
path_covid19 = f'{path}/covid19-en.csv'

data = pd.read_csv(path_tennis)
data

,weather,temperature,humidity,wind,play
0,sunny,high,high,no,no
1,sunny,high,high,yes,no
2,overcast,high,high,no,yes
3,rain,medium,high,no,yes
4,rain,low,normal,no,yes
5,rain,low,normal,yes,no
6,overcast,low,normal,yes,yes
7,sunny,medium,high,no,no
8,sunny,low,normal,no,yes
9,rain,medium,normal,no,yes


# Ejemplo 1: TENIS

## Paso 1

- En la versión Naive se aplica el teorema de Bayes considerando las de entrada variables como independientes para facilitar los cálculos.
- Básicamente vamos a realizar un sistema para contar las repeticiones de una categoría de una variable de entrada respecto de la objetivo.
- Con las repeticiones anteriores se aplicará el teorema de Bayes para calcular la probabilidad condicional de pertenecia a las clases de la variable objetivo y así clasificar una entrada dada.

In [4]:
y_name = data.columns[-1]     # El nombre de la última columna
X_names = data.columns[:-1]   # El nombre de todas las variables hasta la última columna (excluida)

y_classes = data[y_name].unique() # Las clases que se tienen que predecir
N = len(data)                     # Número de muestras

print('target', y_name, y_classes)
print('Input variables', X_names)
print('Number of examples', N)

target play ['no' 'yes']
Input variables Index(['weather', 'temperature', 'humidity', 'wind'], dtype='object')
Number of examples 14


### Probabilidad a Priory

¿Qué probabilidad tenemos de acertar sin información adicional?
 Hay que contar cuántas veces se repiten las clases de la variable objetivo y normalizar.

In [5]:
P_y = pd.DataFrame(data[y_name].value_counts())
P_y['prob'] = data[y_name].value_counts(normalize=True)
P_y

,play,prob
yes,9,0.642857
no,5,0.357143


## Paso 2



Supongamos que la entrada $X$ esta formada únicamente por la primera variable. Las probabilidades las calcularíamos como sigue:

In [6]:
x_name = X_names[0] # 'Weather' o 'Fever' según el conjunto de datos elegido (variables de entrada).

# Contar el número de veces que se repite la categoría de la característica según la categoría objetivo
# unstack() sive para mover los valores de la filas (counts) en nuevas columnas
# fillna(0.0) previene los valores nulos cuando no existe repeticiones en una categoría
df_prob = pd.DataFrame(data[[x_name,y_name]].value_counts(), columns=['counts']).unstack().fillna(0.0)
df_prob[[('prob', i) for i in y_classes]] = df_prob / df_prob.sum()
display(df_prob.round(2))

counts      prob      
play         no  yes   no   yes
weather                        
overcast    0.0  4.0  0.0  0.44
rain        2.0  3.0  0.4  0.33
sunny       3.0  2.0  0.6  0.22

Supongamos el caso de que $X=(rain)$ para calcular la probabilidad de que se juegue o no sería calcular $$P(\textrm{play}=\textrm{yes}|\textrm{weather}=\textrm{rain})$$ y $$P(\textrm{play}=\textrm{no}|\textrm{weather}=\textrm{rain})$$

Recordemos la fórmula: $$P(y | x_1, x_2,\dots,x_n) = \frac{P(y) \prod_{n=1}^{n} P(x_i|y)}{\sum_j P(y_j) \prod_{n=1}^{n} P(x_i|y_j)}$$

In [7]:
df_prob.columns, df_prob.index

(MultiIndex([('counts',  'no'),
             ('counts', 'yes'),
             (  'prob',  'no'),
             (  'prob', 'yes')],
            names=[None, 'play']),
 Index(['overcast', 'rain', 'sunny'], dtype='object', name='weather'))

In [8]:
# P(play=yes|weather=rain)
numerator_yes = P_y.loc['yes','prob']*df_prob.loc['rain',('prob','yes')]

# P(play=no|weather=rain)
numerator_no = P_y.loc['no','prob']*df_prob.loc['rain',('prob','no')]

P_si = numerator_yes / (numerator_yes + numerator_no)
P_no = numerator_no / (numerator_yes + numerator_no)

print(f'P(play=yes|weather=rain)={P_si:.4f}')
print(f'P(play=no|weather=rain)={P_no:.4f}')

P(play=yes|weather=rain)=0.6000
P(play=no|weather=rain)=0.4000


- Calcular las repeticiones condicionadas de las categorías de cada variable respecto de las clases de la variable objetivo.

In [9]:
dict_df_prob = {}
for X_name in X_names:
    # Contar el número de veces que se repite la categoría de la característica según la categoría objetivo
    df_counts = pd.DataFrame(data[[X_name, y_name]].value_counts(), columns=['counts']).unstack().fillna(0.0)

    # Transforma el número de veces en probabilidades según las categorías objetivo
    df_prob = df_counts / df_counts.sum()
    
    # Almacenar el DataFrame en el diccionario para su uso posterior
    dict_df_prob[X_name] = df_prob

# Ejemplo de cómo acceder a los datos
for X_name in X_names:
    print(f"Probabilidades condicionales para {X_name}:")
    print(dict_df_prob[X_name])


Probabilidades condicionales para weather:
         counts          
play         no       yes
weather                  
overcast    0.0  0.444444
rain        0.4  0.333333
sunny       0.6  0.222222
Probabilidades condicionales para temperature:
            counts          
play            no       yes
temperature                 
high           0.4  0.222222
low            0.2  0.333333
medium         0.4  0.444444
Probabilidades condicionales para humidity:
         counts          
play         no       yes
humidity                 
high        0.8  0.333333
normal      0.2  0.666667
Probabilidades condicionales para wind:
     counts          
play     no       yes
wind                 
no      0.4  0.666667
yes     0.6  0.333333


## Paso 3

Calcular las probabilidades de las siguientes entradas

In [10]:
X_test_tenis = pd.DataFrame([
    ['overcast', 'low',    'normal', 'yes'],
    ['rain',     'medium', 'normal', 'no' ],
    ['sunny',    'high',   'normal', 'no' ]
  ],
  columns = X_names
)

X_test_covid19 = pd.DataFrame([
    ['Yes', 'Yes', 'No', 'Yes'],
    ['No',  'Yes', 'No', 'No' ],
    ['Yes', 'Yes', 'No', 'No' ]
  ],
  columns = X_names
)
X_test = X_test_tenis
X_test

,weather,temperature,humidity,wind
0,overcast,low,normal,yes
1,rain,medium,normal,no
2,sunny,high,normal,no


In [11]:
df_res = X_test.copy()
for idx, x in X_test.iterrows():
    # Inicializa la probabilidad de cada clase para la instancia actual
    prob_classes = {y: P_y.loc[y, 'prob'] for y in y_classes}

    # Multiplica las probabilidades condicionales de cada característica
    for y in y_classes:
        for feature in X_names:
            feature_value = x[feature]
            # Accede correctamente a las probabilidades condicionales
            if (feature_value, 'counts') in dict_df_prob[feature].columns:
                prob_classes[y] *= dict_df_prob[feature].loc[feature_value, (y, 'counts')]

    # Normaliza las probabilidades para que sumen 1
    total_prob = sum(prob_classes.values())
    for y in y_classes:
        df_res.at[idx, y] = prob_classes[y] / total_prob if total_prob > 0 else 0

# Redondear las probabilidades para mejorar la legibilidad
df_res = df_res.round(4)

df_res
'''
Respuesta:
  no
  0 no 0.3571 [0.  0.2 0.2 0.6]
  yes
  0 yes 0.6429 [0.4444 0.3333 0.6667 0.3333]
  no
  1 no 0.3571 [0.4 0.4 0.2 0.4]
  yes
  1 yes 0.6429 [0.3333 0.4444 0.6667 0.6667]
  no
  2 no 0.3571 [0.6 0.4 0.2 0.4]
  yes
  2 yes 0.6429 [0.2222 0.2222 0.6667 0.6667]
'''

print(df_res)


    weather temperature humidity wind      no     yes
0  overcast         low   normal  yes  0.3571  0.6429
1      rain      medium   normal   no  0.3571  0.6429
2     sunny        high   normal   no  0.3571  0.6429


## Naive Bayes (scikit-learn)

Vamos a obtener el resultado usando `scikit-learn` entrenando un sistema con Naive Bayes y mostrando los resultados.

### APUNTES
`LabelEncoder` es una herramienta de preprocesamiento de scikit-learn que convierte categorías en números. Cada categoría única en una columna se asigna a un número entero único.


In [148]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

# Preprocesamiento para convertir categorías a números
dict_label_encoder = {}
data_codes = data.copy()
feature_names = []  # Lista para almacenar el orden de las características

# 'play' es la columna objetivo
for i in data.columns.difference(['play']):
    le = LabelEncoder()
    
    le.fit(sorted(data[i].unique())) #obtiene todos los valores únicos de la columna.
    
    # Esto convierte todas las categorías en cada columna en números enteros
    data_codes[i] = le.transform(data[i])
    
    # Por si hubiera que revertir la codificación
    dict_label_encoder[i] = le
    # Agregar el nombre de la característica a la lista
    feature_names.append(i)  

# Crear el Modelo
model = GaussianNB()

# Entrenar con los datos correspondientes
X = data_codes[feature_names]  # Características en el orden correcto
y = data_codes['play']  # Objetivo
model.fit(X, y)

# Preprocesar X_test usando los encoders ya creados y en el mismo orden
X_test_codes = pd.DataFrame()
for feature in feature_names:
    X_test_codes[feature] = dict_label_encoder[feature].transform(X_test[feature])

# Realizar las predicciones de X_test
predicted = model.predict(X_test_codes)

# Visualizar las predicciones
predicted


array(['yes', 'yes', 'yes'], dtype='<U3')

# Ejemplo 1: COVID-19

Aplicar los tres pasos anteriores al conjunto de datos de COVID-19

In [149]:
path_covid19 = f'{path}/covid19-en.csv'
data = pd.read_csv(path_covid19)
data

,Id,Fever,Cough,Respiratory Problems,Infected
0,1,No,No,No,No
1,2,Yes,Yes,Yes,Yes
2,3,Yes,Yes,No,No
3,4,Yes,No,Yes,Yes
4,5,Yes,Yes,Yes,Yes
5,6,No,Yes,No,No
6,7,Yes,No,Yes,Yes
7,8,Yes,No,Yes,Yes
8,9,No,Yes,Yes,Yes
9,10,Yes,Yes,No,Yes


## Paso 1

- En la versión Naive se aplica el teorema de Bayes considerando las de entrada variables como independientes para facilitar los cálculos.
- Básicamente vamos a realizar un sistema para contar las repeticiones de una categoría de una variable de entrada respecto de la objetivo.
- Con las repeticiones anteriores se aplicará el teorema de Bayes para calcular la probabilidad condicional de pertenecia a las clases de la variable objetivo y así clasificar una entrada dada.

In [150]:
y_name = data.columns[-1]     # El nombre de la última columna
X_names = data.columns[:-1]   # El nombre de todas las variables hasta la última columna (excluida)

y_classes = data[y_name].unique() # Las clases que se tienen que predecir
N = len(data)                     # Número de muestras

print('target', y_name, y_classes)
print('Input variables', X_names)
print('Number of examples', N)

target Infected ['No' 'Yes']
Input variables Index(['Id', 'Fever', 'Cough', 'Respiratory Problems'], dtype='object')
Number of examples 14


In [151]:
P_y = pd.DataFrame(data[y_name].value_counts())
P_y['prob'] = data[y_name].value_counts(normalize=True)
P_y

,Infected,prob
Yes,8,0.571429
No,6,0.428571


In [152]:
x_name = X_names[0] 

# Contar el número de veces que se repite la categoría de la característica según la categoría objetivo
# unstack() sive para mover los valores de la filas (counts) en nuevas columnas
# fillna(0.0) previene los valores nulos cuando no existe repeticiones en una categoría
df_prob = pd.DataFrame(data[[x_name,y_name]].value_counts(), columns=['counts']).unstack().fillna(0.0)
df_prob[[('prob', i) for i in y_classes]] = df_prob / df_prob.sum()
display(df_prob.round(2))

counts       prob      
Infected     No  Yes    No   Yes
Id                              
1           1.0  0.0  0.17  0.00
2           0.0  1.0  0.00  0.12
3           1.0  0.0  0.17  0.00
4           0.0  1.0  0.00  0.12
5           0.0  1.0  0.00  0.12
6           1.0  0.0  0.17  0.00
7           0.0  1.0  0.00  0.12
8           0.0  1.0  0.00  0.12
9           0.0  1.0  0.00  0.12
10          0.0  1.0  0.00  0.12
11          1.0  0.0  0.17  0.00
12          0.0  1.0  0.00  0.12
13          1.0  0.0  0.17  0.00
14          1.0  0.0  0.17  0.00

In [153]:
df_prob.columns, df_prob.index

(MultiIndex([('counts',  'No'),
             ('counts', 'Yes'),
             (  'prob',  'No'),
             (  'prob', 'Yes')],
            names=[None, 'Infected']),
 Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64', name='Id'))

### PASO 2
Supongamos que la entrada $X$ esta formada únicamente por la primera variable. Las probabilidades las calcularíamos como sigue:

In [154]:
# Excluir la columna 'Id' de las características
X_names = data.columns[1:-1]   # Excluyendo 'Id' y la última columna

# Repetir el proceso para cada característica relevante
for x_name in X_names:
    df_prob = pd.DataFrame(data[[x_name, y_name]].value_counts(), columns=['counts']).unstack().fillna(0.0)
    df_prob[[('prob', i) for i in y_classes]] = df_prob / df_prob.sum()

    # Mostrar las probabilidades condicionales
    display(df_prob.round(2))



counts      prob      
Infected     No Yes    No   Yes
Fever                          
No            4   2  0.67  0.25
Yes           2   6  0.33  0.75

counts      prob      
Infected     No Yes    No   Yes
Cough                          
No            1   3  0.17  0.38
Yes           5   5  0.83  0.62

counts      prob      
Infected                 No Yes    No   Yes
Respiratory Problems                       
No                        5   1  0.83  0.12
Yes                       1   7  0.17  0.88

In [155]:
# Ejemplo para la característica 'Fever'
for x_name in X_names:
    # Calcula las probabilidades condicionales para 'Fever = Yes'
    numerator_yes = P_y.loc['Yes', 'prob'] * df_prob.loc['Yes', ('prob', 'Yes')]
    numerator_no = P_y.loc['No', 'prob'] * df_prob.loc['Yes', ('prob', 'No')]

    # Calcula las probabilidades condicionales normalizadas
    P_yes = numerator_yes / (numerator_yes + numerator_no)
    P_no = numerator_no / (numerator_yes + numerator_no)

    print(f'P(Infected=Yes|Fever=Yes)={P_yes:.4f}')
    print(f'P(Infected=No|Fever=Yes)={P_no:.4f}')

P(Infected=Yes|Fever=Yes)=0.8750
P(Infected=No|Fever=Yes)=0.1250
P(Infected=Yes|Fever=Yes)=0.8750
P(Infected=No|Fever=Yes)=0.1250
P(Infected=Yes|Fever=Yes)=0.8750
P(Infected=No|Fever=Yes)=0.1250


## Paso 3

Calcular las probabilidades de las siguientes entradas

In [156]:
X_test_covid19 = pd.DataFrame([
    ['Yes', 'Yes', 'No'],
    ['No',  'Yes', 'No'],
    ['Yes', 'Yes', 'No']
  ],
  columns = X_names  # Asegúrate de que X_names solo incluye las características
)
X_test = X_test_covid19
print(X_test)


dict_df_prob = {}
for feature in X_names:
    # Calcula el conteo de cada combinación de valor de característica y clase objetivo
    df_counts = pd.DataFrame(data[[feature, y_name]].value_counts(), columns=['counts']).unstack().fillna(0.0)

    # Calcula las probabilidades condicionales
    df_prob = df_counts / df_counts.sum()

    # Almacena el resultado en dict_df_prob
    dict_df_prob[feature] = df_prob

# Ahora, puedes realizar los cálculos de Naive Bayes con este diccionario
df_res = X_test.copy()
for idx, x in X_test.iterrows():
    # Inicializa la probabilidad de cada clase para la instancia actual
    prob_classes = {y: P_y.loc[y, 'prob'] for y in y_classes}

    # Multiplica las probabilidades condicionales de cada característica
    for y in y_classes:
        for feature in X_names:
            feature_value = x[feature]
            # Asegúrate de acceder correctamente a las probabilidades condicionales
            if (feature_value, 'counts') in dict_df_prob[feature].columns:
                prob_classes[y] *= dict_df_prob[feature].loc[feature_value, (y, 'counts')]

    # Normaliza las probabilidades para que sumen 1
    total_prob = sum(prob_classes.values())
    for y in y_classes:
        df_res.at[idx, y] = prob_classes[y] / total_prob if total_prob > 0 else 0

# Redondear las probabilidades para mejorar la legibilidad
df_res = df_res.round(4)

print(df_res)


  Fever Cough Respiratory Problems
0   Yes   Yes                   No
1    No   Yes                   No
2   Yes   Yes                   No
  Fever Cough Respiratory Problems      No     Yes
0   Yes   Yes                   No  0.4286  0.5714
1    No   Yes                   No  0.4286  0.5714
2   Yes   Yes                   No  0.4286  0.5714
